I'm a beginner, so please be gentle with me.

In [1]:
import os
os.chdir('C:/Users/Takanori/Desktop/Kaggle/TPS2205/input')

# DataSet & Library Loading

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time, gc
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
from torch.optim import lr_scheduler
import string

In [3]:
os.makedirs('model', exist_ok=True)

In [4]:
class CFG:
    label = 'target'  # 目標変数
    batch_size = 10_000  # バッチサイズ
    apex=True  # 混合精度を使い、float16でも行けるようにする
    seed=42
    n_fold = 5
    max_grad_norm=1000
    print_batch_freq=100
    print_epoch_freq = 1
    num_workers=0
    trn_fold=[0,1,2,3,4]
    lr=1e-2
    lr_min=1e-4
    lr_gamma=0.9
    epochs=100
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    num_emb=128

## FE

In [5]:
# データセット読み込み
train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")

In [6]:
df_all = pd.concat([train, test])

In [7]:
df_all['i_02_21'] = (df_all.f_21 + df_all.f_02 > 5.2).astype(int) - (df_all.f_21 + df_all.f_02 < -5.3).astype(int)
df_all['i_05_22'] = (df_all.f_22 + df_all.f_05 > 5.1).astype(int) - (df_all.f_22 + df_all.f_05 < -5.4).astype(int)
i_00_01_26 = df_all.f_00 + df_all.f_01 + df_all.f_26
df_all['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

In [8]:
# NA埋め
df_all.fillna(df_all.mean(numeric_only=True), inplace=True)

In [9]:
# f_27を分解
def splitter(text):
    arr = tuple(text)
    return arr

column_list = ['f_27_' + str(i) for i in range(10)]
df_all[column_list] = df_all.apply(lambda x: splitter(x['f_27']), axis=1, result_type='expand')

In [10]:
# f_27_0~9を数値に
for column in column_list:
    df_all[column] = df_all[column].apply(lambda x: ord(x)-ord('A'))

In [11]:
# f_27に出てくる文字が、何種類あるかを返す
def calc_cnt_of(txt):
    cnt_of = Counter(list(txt))
    return len(cnt_of)

df_all['len_cnt_of'] = df_all['f_27'].apply(calc_cnt_of)

In [12]:
# f_27に各文字が何回出てくるかを数える
def calc_char_cnt(txt):
    cnt_of = Counter(list(txt))
    ret = []
    for c in string.ascii_uppercase:
        ret.append(cnt_of[c])
    return tuple(ret)


column_list = []
for c in string.ascii_uppercase:
    column_list.append('f_27_cnt_' + c)

df_all[column_list] = df_all.apply(lambda x: calc_char_cnt(x['f_27']), axis=1, result_type='expand')

In [13]:
# 不要になったカテゴリ変数を削除
df_all.drop(['f_27'], axis=1, inplace=True)

In [14]:
cat_features = []
val_features = []

In [15]:
for column in df_all.columns:
    if column in ['id', CFG.label]:
        continue
    # int型であり、num_emb未満の種類数しかない場合は、cat_featuresに追加
    if df_all[column].dtype == 'int64' and df_all[column].nunique() < CFG.num_emb:
        cat_features.append(column)
    else:
        val_features.append(column)
print(cat_features)
print(val_features)

['f_07',
 'f_08',
 'f_09',
 'f_10',
 'f_11',
 'f_12',
 'f_13',
 'f_14',
 'f_15',
 'f_16',
 'f_17',
 'f_18',
 'f_29',
 'f_30',
 'f_27_0',
 'f_27_1',
 'f_27_2',
 'f_27_3',
 'f_27_4',
 'f_27_5',
 'f_27_6',
 'f_27_7',
 'f_27_8',
 'f_27_9',
 'len_cnt_of',
 'f_27_cnt_A',
 'f_27_cnt_B',
 'f_27_cnt_C',
 'f_27_cnt_D',
 'f_27_cnt_E',
 'f_27_cnt_F',
 'f_27_cnt_G',
 'f_27_cnt_H',
 'f_27_cnt_I',
 'f_27_cnt_J',
 'f_27_cnt_K',
 'f_27_cnt_L',
 'f_27_cnt_M',
 'f_27_cnt_N',
 'f_27_cnt_O',
 'f_27_cnt_P',
 'f_27_cnt_Q',
 'f_27_cnt_R',
 'f_27_cnt_S',
 'f_27_cnt_T',
 'f_27_cnt_U',
 'f_27_cnt_V',
 'f_27_cnt_W',
 'f_27_cnt_X',
 'f_27_cnt_Y',
 'f_27_cnt_Z']

['f_00',
 'f_01',
 'f_02',
 'f_03',
 'f_04',
 'f_05',
 'f_06',
 'f_19',
 'f_20',
 'f_21',
 'f_22',
 'f_23',
 'f_24',
 'f_25',
 'f_26',
 'f_28',
 'i_02_21',
 'i_05_22',
 'i_00_01_26']

In [16]:
train = df_all[:len(train)]
test = df_all[len(train):]

## CV

In [17]:
# CV Split
skf = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(skf.split(train, train[CFG.label])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

C:\Users\Takanori\AppData\Local\Temp\ipykernel_24652\998641302.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[val_index, 'fold'] = int(n)
C:\Users\Takanori\AppData\Local\Temp\ipykernel_24652\998641302.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['fold'] = train['fold'].astype(int)


fold
0    180000
1    180000
2    180000
3    180000
4    180000
dtype: int64

# Pytorch

## Define DateLoader

In [18]:
class TrainDataset(Dataset):
    def __init__(self, df, cat_features, val_features, target):
        self.cat_data = df[cat_features].values
        self.val_data = df[val_features].values
        self.target = df[target].values

    def __len__(self):
        return len(self.target)

    def __getitem__(self, item):
        cat_inputs = torch.tensor(self.cat_data[item], dtype=torch.long)
        val_inputs = torch.tensor(self.val_data[item], dtype=torch.float32)
        labels = torch.tensor(self.target[item], dtype=torch.float32)
        return cat_inputs, val_inputs, labels

## Define Pytorch Model

In [19]:
class EarlyStopping:
    def __init__(self, window):
        from collections import deque
        self.que1 = deque()
        self.que2 = deque()
        self.window = window

    def step(self, loss):
        self.que1.append(loss)
        if len(self.que1) > self.window:
            rm = self.que1.popleft()
            self.que2.append(rm)
        if len(self.que2) > self.window:
            rm = self.que2.popleft()
        # que1が最近のloss, que2が過去のloss
        # 最近のlossのほうが高い場合は、処理を停止させるためにTrueを返す
        if len(self.que2) < self.window:
            return False
        if sum(self.que1) > sum(self.que2):
            print(self.que1)
            print(self.que2)
            return True
        return False

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [21]:
class Net1(nn.Module):
    def __init__(self, fold):
        super(Net1, self).__init__()

        self.first_bn = nn.BatchNorm1d(len(val_features), momentum=0.01)  # とりあえず入れてみた。momentumが小さいといいことあるんかな・・？
        self.fc1 = nn.Linear(len(val_features)+len(cat_features)*1, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 384)
        self.bn2 = nn.BatchNorm1d(384)
        self.fc3 = nn.Linear(384, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        self.fc6 = nn.Linear(64, 32)
        self.bn6 = nn.BatchNorm1d(32)
        self.fc7 = nn.Linear(32, 1)
        self.bn7 = nn.BatchNorm1d(1)
        emb_list = [nn.Embedding(CFG.num_emb, 1) for _ in range(len(cat_features))]
        self.emb_list = nn.ModuleList(emb_list)

        self.prepare_data(fold)
        self.criterion = nn.BCEWithLogitsLoss()
        self.device = CFG.device
        self.configure_optimizer()

    def forward(self, cat_inputs, val_inputs):
        # dropoutの後にbnを置いてはならない
        # nbは、活性化関数の前に置く
        x0 = self.first_bn(val_inputs)
        embs = [self.emb_list[i](cat_inputs[:, i]) for i in range(len(cat_features))]
        x1 = torch.cat(embs, dim=1)
        x1 = x1.view(x1.size(0), -1)
        x = torch.cat([x0, x1], dim=1)
        x = F.silu(self.bn1((self.fc1(x))))
        x = F.silu(self.bn2((self.fc2(x))))
        x = F.silu(self.bn3((self.fc3(x))))
        x = F.silu(self.bn4((self.fc4(x))))
        x = F.silu(self.bn5((self.fc5(x))))
        x = F.silu(self.bn6((self.fc6(x))))
        x = self.bn7((self.fc7(x)))
        return x

    # Setup DataLoaders
    def prepare_data(self, fold):
        train_folds = train[train['fold'] != fold].reset_index(drop=True)  # 参照が消えたりする？しないか。
        valid_folds = train[train['fold'] == fold].reset_index(drop=True)
        self.train_dataset = TrainDataset(train_folds, cat_features, val_features, CFG.label)
        self.valid_dataset = TrainDataset(valid_folds, cat_features, val_features, CFG.label)
        self.valid_labels = valid_folds[CFG.label].values
        self.test_dataset = TrainDataset(test, cat_features, val_features, CFG.label)
        self.test_labels = test[CFG.label].values

    def train_dataloader(self):
        data_loader = DataLoader(self.train_dataset,
                                batch_size=CFG.batch_size,
                                shuffle=True,
                                num_workers=CFG.num_workers,
                                pin_memory=True,
                                drop_last=True)
        return data_loader

    def valid_dataloader(self):
        data_loader = DataLoader(self.valid_dataset,
                                batch_size=CFG.batch_size,
                                shuffle=False,
                                num_workers=CFG.num_workers,
                                pin_memory=True,
                                drop_last=False)
        return data_loader

    def test_dataloader(self):
        data_loader = DataLoader(self.test_dataset,
                                batch_size=CFG.batch_size,
                                shuffle=False,
                                num_workers=CFG.num_workers,
                                pin_memory=True,
                                drop_last=False)
        return data_loader

    # Setup Optimizer and Scheduler
    def configure_optimizer(self):
        optimizer = optim.Adam(self.parameters(), lr=CFG.lr)

        # CFG.lrからCFG.lr_minに収束させる
        num_train_steps = len(self.train_dataset) // CFG.batch_size * CFG.epochs
        n = math.log(CFG.lr_min) / math.log(CFG.lr_gamma)
        step_size = math.ceil(num_train_steps / n)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=CFG.lr_gamma)
        self.optimizer = optimizer
        self.scheduler = scheduler

    # training valid test steps
    def training_step(self):
        losses = AverageMeter()
        for step, (cat_inputs, val_inputs, labels) in enumerate(self.train_dataloader()):
            cat_inputs, val_inputs, labels = cat_inputs.to(self.device), val_inputs.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            outputs = self(cat_inputs, val_inputs).squeeze()
            loss = self.criterion(outputs, labels)
            losses.update(loss.item(), cat_inputs.size(0))
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
        return losses.avg

    def validation_step(self):
        losses = AverageMeter()
        preds = []
        with torch.no_grad():
            for step, (cat_inputs, val_inputs, labels) in enumerate(self.valid_dataloader()):
                cat_inputs, val_inputs, labels = cat_inputs.to(self.device), val_inputs.to(self.device), labels.to(self.device)
                outputs = self(cat_inputs, val_inputs).squeeze()
                loss = self.criterion(outputs, labels)
                losses.update(loss.item(), cat_inputs.size(0))
                outputs = torch.sigmoid(outputs)
                preds.append(outputs.cpu().numpy())
        return losses.avg, np.concatenate(preds, axis=0)

    def test_step(self):
        preds = []
        with torch.no_grad():
            for step, (cat_inputs, val_inputs, labels) in enumerate(self.test_dataloader()):
                cat_inputs, val_inputs, labels = cat_inputs.to(self.device), val_inputs.to(self.device), labels.to(self.device)
                outputs = self(cat_inputs, val_inputs).squeeze()
                outputs = torch.sigmoid(outputs)  # 予測時は活性化関数を書ける
                preds.append(outputs.cpu().numpy())
        return np.concatenate(preds, axis=0)

# Pytorch Training

In [22]:
for fold in range(CFG.n_fold):
    print('='*10, 'fold', fold, '='*10)

    model = Net1(fold).to(CFG.device)
    best_loss = float('inf')
    es = EarlyStopping(window=10)

    for epoch in range(CFG.epochs):
        model.train()
        train_loss = model.training_step()

        model.eval()
        valid_loss, preds = model.validation_step()
        auc = roc_auc_score(model.valid_labels, preds)
        if epoch % CFG.print_epoch_freq == 0:
            print('Epoch: {}/{}'.format(epoch, CFG.epochs), '\tTrain Loss: {:.4f}'.format(train_loss), '\tValid Loss: {:.4f}'.format(valid_loss), '\tLR: {:.4f}'.format(model.optimizer.param_groups[0]['lr']), '\tAUC: {:.4f}'.format(auc))

        if best_loss > valid_loss:
            print('best loss updated: {:.4f} -> {:.4f}'.format(best_loss, valid_loss), 'saving model...')
            best_loss = valid_loss
            torch.save(model.state_dict(), './model/model_fold_{}.pth'.format(fold))
        if es.step(valid_loss):
            print('early stopping')
            break
    gc.collect()
    torch.cuda.empty_cache()
# TODO float16でも動くようにする
# TODO emb_dimはいまいち、num_embに変更する

========== fold 0 ==========
Epoch: 0/1000 	Train Loss: 0.3731 	Valid Loss: 0.3308 	LR: 0.0100 	AUC: 0.9474
best loss updated: inf -> 0.3308 saving model...
Epoch: 1/1000 	Train Loss: 0.2144 	Valid Loss: 0.2455 	LR: 0.0100 	AUC: 0.9688
best loss updated: 0.3308 -> 0.2455 saving model...
Epoch: 2/1000 	Train Loss: 0.1534 	Valid Loss: 0.1693 	LR: 0.0100 	AUC: 0.9877
best loss updated: 0.2455 -> 0.1693 saving model...
Epoch: 3/1000 	Train Loss: 0.1215 	Valid Loss: 0.1358 	LR: 0.0100 	AUC: 0.9922
best loss updated: 0.1693 -> 0.1358 saving model...
Epoch: 4/1000 	Train Loss: 0.1030 	Valid Loss: 0.1055 	LR: 0.0100 	AUC: 0.9951
best loss updated: 0.1358 -> 0.1055 saving model...
Epoch: 5/1000 	Train Loss: 0.0898 	Valid Loss: 0.0920 	LR: 0.0100 	AUC: 0.9963
best loss updated: 0.1055 -> 0.0920 saving model...
Epoch: 6/1000 	Train Loss: 0.0808 	Valid Loss: 0.0858 	LR: 0.0100 	AUC: 0.9966
best loss updated: 0.0920 -> 0.0858 saving model...
Epoch: 7/1000 	Train Loss: 0.0736 	Valid Loss: 0.0801 	LR

KeyboardInterrupt: 

# predictions

In [ ]:
import glob

In [ ]:
models = np.sort(glob.glob(f"./model/*.pth"))
print(models)
# fold別に作った10個のモデルをロードする

['./model\\model_fold_0.pth' './model\\model_fold_1.pth'
 './model\\model_fold_2.pth' './model\\model_fold_3.pth'
 './model\\model_fold_4.pth']


In [ ]:
predictions = pd.DataFrame()
with torch.no_grad():
    for i, model_name in enumerate(models):
        model = Net1(i, len(all_features), 1).to(CFG.device)
        model.load_state_dict(torch.load(model_name,))
        pred = model.test_step()
        predictions = pd.concat([predictions, pd.DataFrame(pred)], axis=1)

predictions

[[0.9752137 ]
 [0.9764059 ]
 [0.02119529]
 ...
 [0.01168523]
 [0.0249472 ]
 [0.02150796]]


,0
0,0.975214
1,0.976406
2,0.021195
3,0.019454
4,0.974685
...,...
699995,0.317303
699996,0.976512
699997,0.011685
699998,0.024947


[[0.9763256 ]
 [0.9798013 ]
 [0.02124067]
 ...
 [0.03036187]
 [0.02803684]
 [0.02402741]]


,0
0,0.976326
1,0.979801
2,0.021241
3,0.019621
4,0.976439
...,...
699995,0.943249
699996,0.976103
699997,0.030362
699998,0.028037


[[0.977869  ]
 [0.9778541 ]
 [0.02024253]
 ...
 [0.03639641]
 [0.02660423]
 [0.02752425]]


...

[[0.9735782 ]
 [0.9757076 ]
 [0.02386909]
 ...
 [0.02657172]
 [0.02519219]
 [0.0249288 ]]


,0
0,0.973578
1,0.975708
2,0.023869
3,0.021889
4,0.976312
...,...
699995,0.777773
699996,0.975972
699997,0.026572
699998,0.025192


[[0.97171324]
 [0.97452515]
 [0.02127126]
 ...
 [0.02477803]
 [0.02724473]
 [0.02345986]]


,0
0,0.971713
1,0.974525
2,0.021271
3,0.020475
4,0.976128
...,...
699995,0.746653
699996,0.976701
699997,0.024778
699998,0.027245


,0,0,0,0,0
0,0.975214,0.976326,0.977869,0.973578,0.971713
1,0.976406,0.979801,0.977854,0.975708,0.974525
2,0.021195,0.021241,0.020243,0.023869,0.021271
3,0.019454,0.019621,0.022278,0.021889,0.020475
4,0.974685,0.976439,0.976386,0.976312,0.976128
...,...,...,...,...,...
699995,0.317303,0.943249,0.866609,0.777773,0.746653
699996,0.976512,0.976103,0.974557,0.975972,0.976701
699997,0.011685,0.030362,0.036396,0.026572,0.024778
699998,0.024947,0.028037,0.026604,0.025192,0.027245


In [ ]:
sub[CFG.label] = predictions.mean(axis=1)

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head(10)

,id,target
0,900000,0.974940
1,900001,0.976859
2,900002,0.021564
3,900003,0.020744
4,900004,0.975990
5,900005,0.025832
6,900006,0.373095
7,900007,0.976420
8,900008,0.022587
9,900009,0.976356
